In [1]:
from yolo_detector import load_yolo_model, detect_image , draw_boxes
from util import write_results, prepare_image, get_detection_summary
from object_detectors_attacks import targeted_dag_attack, disappearance_dag_attack
import torch
from torchvision.utils import save_image

In [2]:
cfgfile = "cfg/yolov3.cfg"
reso = 416
weightsfile = "yolov3.weights"
classes_names = "data/coco.names"
images = "coco/test_images/000000000001.jpg"
batch_size = 1
confidence = 0.5
nms_thesh = 0.4
CUDA = torch.cuda.is_available()
det = "detections"

In [3]:
model, classes, num_classes, inp_dim = load_yolo_model(cfgfile,weightsfile,classes_names, reso ,CUDA = CUDA)

Loading network.....


Network successfully loaded


In [4]:
im_batches, im_dim_list, imlist, loaded_ims = prepare_image(images, inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, im_batches, im_dim_list, imlist, loaded_ims, batch_size, confidence, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, det)
target_objects = get_detection_summary(output)
print(target_objects)

----------------------------------------------------------
Objects Detected:    car car truck parking meter
----------------------------------------------------------
Detection images saved in detections with prefix: _
[2, 12, 7]


In [7]:
adversarial_class = [4,5,6] #aeroplane, bus, train #the adversarial classes are relevant to the target objects
adv_images_dag, _ = targeted_dag_attack(
    model, 
    images, 
    target_objects, 
    adversarial_class, 
    max_iter=20, 
    gamma=0.1, 
    output_dir="targeted_dag_attack_adversarial_images",
    im_name_prefix="relevalant_adversarial_objects",
    CUDA=CUDA
)


Batch 1 | Iter 1/20:


  Detected classes: [2, 2, 7, 12]
  Current loss: -2489.4719

Batch 1 | Iter 2/20:
  Detected classes: [2, 7]
  Current loss: -408.2347

Batch 1 | Iter 3/20:
  Detected classes: [2, 7]
  Current loss: -114.5922

Batch 1 | Iter 4/20:
  Detected classes: [0, 6, 7]
  Current loss: 14.7611

Batch 1 | Iter 5/20:
  Detected classes: [0, 0, 0, 0, 6, 6, 12]
  Current loss: 22.9750

Batch 1 | Iter 6/20:
  Detected classes: [0, 5, 6, 9]
  Boosting undetected classes: [4]
  Current loss: 119.2097

Batch 1 | Iter 7/20:
  Detected classes: [0, 5, 6, 9]
  Boosting undetected classes: [4]
  Current loss: 399.1396

Batch 1 | Iter 8/20:
  Detected classes: [5, 6, 7]
  Current loss: 21.9280

Batch 1 | Iter 9/20:
  Detected classes: [5, 6, 9]
  Boosting undetected classes: [4]
  Current loss: 212.3640

Batch 1 | Iter 10/20:
  Detected classes: [5, 6, 7, 9]
  Current loss: 284.4173

Batch 1 | Iter 11/20:
  Detected classes: [5, 6]
  Boosting undetected classes: [4]
  Current loss: 563.3346

Batch 1 | Iter

In [9]:
adv_batch_dag, im_dim_list, imlist, loaded_ims = prepare_image("targeted_dag_attack_adversarial_images", inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, adv_batch_dag, im_dim_list, imlist, loaded_ims, batch_size, confidence, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "targeted_dag_adversarial_detections","relevant")
  

----------------------------------------------------------
Objects Detected:    aeroplane bus train
----------------------------------------------------------
Detection images saved in targeted_dag_adversarial_detections with prefix: relevant_


In [10]:
#apply the targeted DAG attack to the image but with irrelevant adversarial classes
adversarial_class = [33,34,35] #kite, baseball bat, baseball glove
adv_images_dag, _ = targeted_dag_attack(
    model, 
    images, 
    target_objects, 
    adversarial_class, 
    max_iter=20, 
    gamma=0.1, 
    output_dir="targeted_dag_attack_adversarial_images",
    im_name_prefix="irrelevalant_adversarial_objects",
    CUDA=CUDA
)


Batch 1 | Iter 1/20:


  Detected classes: [2, 2, 7, 12]
  Current loss: -2515.7395

Batch 1 | Iter 2/20:
  Detected classes: [2, 7]
  Current loss: -457.6368

Batch 1 | Iter 3/20:
  Detected classes: [2, 7, 58]
  Current loss: -152.7484

Batch 1 | Iter 4/20:
  Detected classes: [0, 0, 0, 6, 7, 8]
  Current loss: -53.4850

Batch 1 | Iter 5/20:
  Detected classes: [0, 0, 5, 8]
  Boosting undetected classes: [33, 34, 35]
  Current loss: 164.7587

Batch 1 | Iter 6/20:
  Detected classes: [0, 0, 0, 5, 8]
  Boosting undetected classes: [33, 34, 35]
  Current loss: 525.6567

Batch 1 | Iter 7/20:
  Detected classes: [0, 0, 0, 5, 8, 35]
  Boosting undetected classes: [33, 34]
  Current loss: 1108.9956

Batch 1 | Iter 8/20:
  Detected classes: [0, 0, 2, 5, 7, 8]
  Current loss: 1067.0724

Batch 1 | Iter 9/20:
  Detected classes: [0, 0, 2, 5, 8, 35]
  Current loss: 1584.4465

Batch 1 | Iter 10/20:
  Detected classes: [0, 0, 0, 5, 8, 34, 35]
  Boosting undetected classes: [33]
  Current loss: 2052.7600

Batch 1 | Iter 

In [11]:
#trying larger iterations number to see if the attack can be successful
adversarial_class = [33,34,35] #kite, baseball bat, baseball glove
adv_images_dag, _ = targeted_dag_attack(
    model, 
    images, 
    target_objects, 
    adversarial_class, 
    max_iter=40, 
    gamma=0.1, 
    output_dir="targeted_dag_attack_adversarial_images",
    im_name_prefix="irrelevalant_adversarial_objects",
    CUDA=CUDA
)


Batch 1 | Iter 1/40:
  Detected classes: [2, 2, 7, 12]
  Current loss: -2515.7395

Batch 1 | Iter 2/40:
  Detected classes: [2, 7]
  Current loss: -457.6368

Batch 1 | Iter 3/40:
  Detected classes: [2, 7, 58]
  Current loss: -152.7484

Batch 1 | Iter 4/40:
  Detected classes: [0, 0, 0, 6, 7, 8]
  Current loss: -53.4850

Batch 1 | Iter 5/40:
  Detected classes: [0, 0, 5, 8]
  Boosting undetected classes: [33, 34, 35]
  Current loss: 164.7587

Batch 1 | Iter 6/40:
  Detected classes: [0, 0, 0, 5, 8]
  Boosting undetected classes: [33, 34, 35]
  Current loss: 525.6567

Batch 1 | Iter 7/40:
  Detected classes: [0, 0, 0, 5, 8, 35]
  Boosting undetected classes: [33, 34]
  Current loss: 1108.9956

Batch 1 | Iter 8/40:
  Detected classes: [0, 0, 2, 5, 7, 8]
  Current loss: 1067.0724

Batch 1 | Iter 9/40:
  Detected classes: [0, 0, 2, 5, 8, 35]
  Current loss: 1584.4465

Batch 1 | Iter 10/40:
  Detected classes: [0, 0, 0, 5, 8, 34, 35]
  Boosting undetected classes: [33]
  Current loss: 2052

In [12]:
#detect the adversarial images
adv_batch_dag, im_dim_list, imlist, loaded_ims = prepare_image("targeted_dag_attack_adversarial_images/irrelevalant_adversarial_objects_adv_0000.png", inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, adv_batch_dag, im_dim_list, imlist, loaded_ims, batch_size, confidence, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "targeted_dag_adversarial_detections","irrelevant")

----------------------------------------------------------
Objects Detected:    person person kite baseball bat baseball bat baseball glove
----------------------------------------------------------
Detection images saved in targeted_dag_adversarial_detections with prefix: irrelevant_


In [13]:
#trying the disappearance attack
adv_images_disappearance, _ = disappearance_dag_attack(
    model, 
    images, 
    target_objects, 
    max_iter=20, 
    gamma=0.1, 
    output_dir="disappearance_attack_adversarial_images",
    im_name_prefix="",
    CUDA=CUDA
)

  Detected classes: [2, 2, 7, 12]
Step 1/20: Loss = -776.558837890625
  Detected classes: [2, 7, 12]
Step 2/20: Loss = -605.2846069335938
  Detected classes: [2, 7, 12]
Step 3/20: Loss = -576.605712890625
  Detected classes: [2, 7, 12]
Step 4/20: Loss = -556.0577392578125
  Detected classes: [2, 2, 7, 12]
Step 5/20: Loss = -664.4259033203125
  Detected classes: [2, 5, 12]
Step 6/20: Loss = -391.04327392578125
  Detected classes: [2, 5, 7, 12]
Step 7/20: Loss = -517.28125
  Detected classes: [2, 5, 7]
Step 8/20: Loss = -314.5614013671875
  Detected classes: [2, 5, 7]
Step 9/20: Loss = -319.78509521484375
  Detected classes: [2, 5, 7, 12]
Step 10/20: Loss = -496.9554138183594
  Detected classes: [0, 0, 2, 2, 5, 7]
Step 11/20: Loss = -459.8917541503906
  Detected classes: [0, 2, 2, 5, 7]
Step 12/20: Loss = -433.4477844238281
  Detected classes: [0, 2, 5, 7]
Step 13/20: Loss = -279.6141662597656
  Detected classes: [0, 5]
Iteration 14: Attack succeeded. Stopping early.
Saved adversarial im

In [14]:
#detect the adversarial images
adv_batch_disappearance, im_dim_list, imlist, loaded_ims = prepare_image("disappearance_attack_adversarial_images", inp_dim)
output, imlist, loaded_ims = detect_image(model, classes, num_classes, inp_dim, adv_batch_disappearance, im_dim_list, imlist, loaded_ims, batch_size, confidence, nms_thesh, CUDA = CUDA)
draw_boxes(loaded_ims,output,imlist,classes, "disappearance_adversarial_detections")

----------------------------------------------------------
Objects Detected:    person bus
----------------------------------------------------------
Detection images saved in disappearance_adversarial_detections with prefix: _
